In [1]:
import os
%pwd

'h:\\Projects\\TextSummarizer\\research'

In [2]:
os.chdir("../")
%pwd

'h:\\Projects\\TextSummarizer'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [22]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        return DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

In [23]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [24]:
class DataValidation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        #tokenizer_name = config.tokenizer_name.replace("\\", "/")
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, examples):
        input_embeddings = self.tokenizer(examples['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_embeddings = self.tokenizer(examples['summary'], max_length=1024, truncation=True)

        return {
            'input_ids': input_embeddings['input_ids'],
            'attention_mask': input_embeddings['attention_mask'],
            'labels': target_embeddings['input_ids']
        }
    
    def convert_data(self):
        samsum_dataset = load_from_disk(self.config.data_path)
        samsum_dataset_pt = samsum_dataset.map(self.convert_examples_to_features, batched=True)
        samsum_dataset_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))

In [25]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataValidation(data_transformation_config)
    data_transformation.convert_data()
except Exception as e:
    raise e

[2024-11-20 13:16:49,907] 16 TextSummarizerLogger - INFO - yaml file config\config.yaml read successfully.
[2024-11-20 13:16:49,910] 16 TextSummarizerLogger - INFO - yaml file params.yaml read successfully.
[2024-11-20 13:16:49,911] 35 TextSummarizerLogger - INFO - Directory artifacts already exists.
[2024-11-20 13:16:49,912] 35 TextSummarizerLogger - INFO - Directory artifacts/data_transformation already exists.


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 54554.63 examples/s]
